In [ ]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


This guide will give you an example of how to conduct a parameter study with
all the necessary steps.  

# 1 Initialization of solver, processor and workflow
We start with initializing of the workflow

In [ ]:
BoSSSshell.WorkflowMgm.Init("Name of Workflow");

This line helps us manage the sessions later on while evaluating the results. 
Next, we connect to the database.

In [ ]:
var myDb = CreateTempDatabase();

To check all the sessions in the current workflow, use the line:

In [ ]:
BoSSSshell.WorkflowMgm.Sessions

Now, all the necessary libraries need to be loaded

In [ ]:
using System.Diagnostics;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.XdgTimestepping;

As an execution queue, we select the first queue defined in 
the **$\sim$/.BoSSS/etc/BatchProcessorConfig.json**-file:

In [ ]:
var myBatch = ExecutionQueues[0];

# 2 Grid Generation
Firstly, we need to determine the boundaries of our grid/control volume. 

Is it important to know that the number of nodes (in our case $k$) needed are equal to the number of cells $+1$. For instance, for $10$ cells we need $11$ nodes.

In this example we will use the Cartesian $2D$ grid from the database
which requires $x$- and $y$-Nodes. The J term in the code is for doing
a check if the desired resolution of the volume is correctly typed.

In [ ]:
int k = 10;
double[] xNodes = GenericBlas.Linspace(0, 1, k + 1);
double[] yNodes = GenericBlas.Linspace(0, 1, k + 1);
int J           = (xNodes.Length - 1)*(yNodes.Length - 1);
string GridName = string.Format(BoSSSshell.WorkflowMgm.CurrentProject + "_J" +J);
 
Console.WriteLine("Creating grid with " + J + " cells. ");
 
GridCommons g;
g      = Grid2D.Cartesian2DGrid(xNodes, yNodes);
g.Name = GridName;

# 3 Define geometrical boundaries

After loading the grid and giving the dimensions, we need to adjust
the edges and their names. 

With the following code we assign every edge with
a number and name. Keep in mind that the name corresponds to the boundary condition (in this case "Pressure Dirichlet").

In this particular case we will use inflow profile represented
via tan-function and the angle of inflow will be $30$ degrees.

In [ ]:
GridCommons g;
g      = Grid2D.Cartesian2DGrid(xNodes, yNodes);
g.Name = GridName;
 
g.EdgeTagNames.Add(1, "wall");
g.EdgeTagNames.Add(2, "Velocity_Inlet");
g.EdgeTagNames.Add(3, "Pressure_Dirichlet_back");
g.EdgeTagNames.Add(4, "Pressure_Dirichlet_top");
 
g.DefineEdgeTags(delegate (double[] X) {
    byte ret = 0;
    if (Math.Abs(X[1]-(0.0))<= 1.0e-8)
        ret = 1;
    if (Math.Abs(X[0]-(0.0))<= 1.0e-8)
        ret = 2;
    if (Math.Abs(X[1]-(1.0))<= 1.0e-8)
        ret = 3;
    if (Math.Abs(X[0]-(1.0))<= 1.0e-8)
        ret = 4;
    return ret;
 
 });
 myDb.SaveGrid(ref g);

# 4 Angle/Velocity Profile
In this particular case we will use inflow profile represented via tan-function and the angle of inflow will be $30$ degrees.

In [ ]:
string caseName = string.Format("k{0}_{1}", k, g);
 
Console.WriteLine("setting up: " + caseName);
 
double beta    = 30;
string CosBeta = Math.Cos(beta*Math.PI/180.0).ToString();
string SinBeta = Math.Sin(beta*Math.PI/180.0).ToString();

These code lines set up the case name and introduce the sine and cosine 
functions to our simulation. 

Next, we define the velocities in 
$x$- and $y$-direction via a tan-function. These velocities and angles are only for this particular example and would not be suited for your simulation.

In [ ]:
var UX = new Formula
    (string.Format("X=> {0}*Math.Atan(X[1]*5)*2.0/Math.PI",CosBeta),false);
var UY = new Formula 
    (string.Format("X=> {0}*Math.Atan(X[1]*5)*2.0/Math.PI",SinBeta),false);

We need to determine the simulation parameters needed to proceed. 

The variable *ctrl* is used to store the **XNSE_Control**-object from which the XNSE Solver will run.
 All other parameters are selfexplanatory.

In [ ]:
var ctrl = new XNSE_Control();
//controls.Add(ctrl);
 
ctrl.SessionName = caseName;
ctrl.SetDatabase(myDb);
ctrl.SetGrid(g);
ctrl.SetDGdegree(k);
ctrl.NoOfMultigridLevels = 5;

Here the velocities and boundary conditions are set. 

In [ ]:
ctrl.AddBoundaryValue("wall");
ctrl.AddBoundaryValue("Velocity_Inlet");
ctrl.AddBoundaryValue("Pressure_Dirichlet_back");
ctrl.AddBoundaryValue("Pressure_Dirichlet_top");
ctrl.AddBoundaryValue("Velocity_Inlet","VelocityX",UX);
ctrl.AddBoundaryValue("Velocity_Inlet","VelocityY",UY);

aswell as the initial values

In [ ]:
ctrl.InitialValues.Add("VelocityX", new Formula ("X=> 0.0", false));
ctrl.InitialValues.Add("VelocityY", new Formula ("X=> 0.0", false));
ctrl.InitialValues.Add("Pressure", new Formula ("X=> 0.0", false));
ctrl.InitialValues.Add("Phi", new Formula ("X=> -1.0", false));

# 5 Fluid properties
Here we set up the density and the Reynolds number,
keep in mind that the calculations are dimensionless, 
so leave the values as seen above ($100$ is an example value)

In [ ]:
double reynolds               = 100;
ctrl.PhysicalParameters.rho_A = 1;
ctrl.PhysicalParameters.mu_A  = 1.0/reynolds;

# 6 Simulation options
We set the simulation parameters, such as time-step size,
end time and number of time-steps.

In [ ]:
ctrl.TimeSteppingScheme = TimeSteppingScheme.ImplicitEuler;
double dt               = 7e-2;
ctrl.dtMax              = dt;
ctrl.dtMin              = dt;
ctrl.Endtime            = 1e16;
ctrl.NoOfTimesteps      = 100;

for the time-stepping scheme, you can choose either BDF2 or ImplicitEuler.

# 7 Starting of simulation
You have two possible ways to start a simulation
- locally on the PC via **myBatch** 
- or on the network cluster **myHPC** (commented) 

In [ ]:
//MiniBatchProcessor.Server.StartIfNotRunning(false)

In [ ]:
 
//Console.WriteLine(" Submitting to Cluster: " + ctrl.SessionName);
//ctrl.RunBatch(myHPC);

Console.WriteLine(" Submitting " + ctrl.SessionName);
ctrl.RunBatch(myBatch);

# 8 Evaluation and Error Calculation

After all of the desired simulation are finished,
you need to evaluate the different parameters and their effect on 
the whole system. Typing the following command gives you a list of all 
simulations with their status (FinishedSuccessful or with certain errors)

In [ ]:
BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Key + ": \t" + kv.Value.Status);

With the next command line you are able to select a certain session(simulation) and see the different time-steps for control purposes.

In [ ]:
BoSSSshell.WorkflowMgm.AllJobs.ElementAt(1).Value.Stdout;

# 9 Evaluation of the $L^2$-Error
This section introduces the calculation of the $L^2$-Error.

In [ ]:
 ITimestepInfo[] AllSolutionS = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value.LatestSession.Timesteps.Last()).ToArray();

In [ ]:
ITimestepInfo[] k1_SolutionS = AllSolutionS.Where(
     ts = > ts.Fields.Single(
           f = > f.Identification == "Pressure").Basis.Degree == 0).ToArray();
ITimestepInfo[] k2_SolutionS = AllSolutionS.Where(
     ts = > ts.Fields.Single(
           f = > f.Identification == "Pressure").Basis.Degree == 1).ToArray();
ITimestepInfo[] k3_SolutionS = AllSolutionS.Where(
     ts = > ts.Fields.Single(
           f = > f.Identification == "Pressure").Basis.Degree == 2).ToArray();

In [ ]:
k1_SolutionS.Select(
     ts => ts.Fields.Single(
           f = > f.Identification == "Pressure").Basis.Degree);

In [ ]:
double[] GridRes;
Dictionary<string, double[]> L2Errors;
DGFieldComparison.ComputeErrors(
     new[]{"VelocityX","VelocityY"}, k1_SolutionS, out GridRes, out L2Errors);

To check the particular errors, type

In [ ]:
GridRes;

In [ ]:
L2Errors["VelocityX"];

In [ ]:
L2Errors["VelocityY"];

# 10 Plotting of errors

This section gives a brief example of how to plot the erros 
and all the data from the previous simulations.

In [ ]:
Plot(GridRes,L2Errors["VelocityX"],"VelXErr","-oy",
     GridRes,L2Errors["VelocityY"],"VelXErr","-xb",logX:true,logY:true);

for a plot with more specifics and more possible adjustments

In [ ]:
var FancyPlot = new Plot2Ddata();

In [ ]:
FancyPlot.LogX = true;
FancyPlot.LogY = true;

In [ ]:
var k1plot = new Plot2Ddata.XYvalues(
    "VelXErr-k1",GridRes,L2Errors["VelocityY"]);

In [ ]:
ArrayTools.AddToArray(k1plot, ref FancyPlot.dataGroups);

In [ ]:
var CL = FancyPlot.ToGnuplot().PlotCairolatex();

In [ ]:
CL.PlotNow();

# 11 Exporting the session table

In [ ]:
static class AddCols {
    static public object SipMatrixAssembly_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("SipMatrixAssembly");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Aggregation_basis_init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Aggregation_basis_init");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_Init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Init");
        var nd  = ndS.ElementAt(0);
        //Console.WriteLine("Number of nodes: " + ndS.Count() + " cc " + nd.CallCount );
        return nd.TimeSpentInMethod.TotalSeconds / nd.CallCount;
    }
    static public object Solver_Run_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Run");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
}

this code adds additional/user-defined colums. Now, we want to export he 
saved session table in a file.

In [ ]:
var SessTab = BoSSSshell.WorkflowMgm.SessionTable;

In [ ]:
SessTab = SessTab.ExtractColumns(AllCols.ToArray());

In [ ]:
using System.IO;

Here, we define the filename

In [ ]:
var now           = DateTime.Now;
SessTab.TableName = "SolverRuns--" + now.Year + "-" + now.Month + "-" + now.Day;
string docpath    = Path.Combine(CurrentDocDir, SessTab.TableName + ".json");

saving the session table as a file could also be done in our git reposatory

In [ ]:
SessTab.SaveToFile(docpath);